In [5]:
import json

import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [6]:
class schema:
    def __init__ (self, id):
        self.id = id
        self.tableNames = []

In [9]:
# Loading Spider Data
with open('./spider-data/tables.json', 'r') as f:
  schemaDoc = json.load(f)

with open('./spider-data/train_spider.json', 'r') as f:
  queryDoc = json.load(f)

schemas = {}

for idx, iSchema in enumerate(schemaDoc):
    dbid = iSchema['db_id']
    test = schema(dbid)
    for columnName in iSchema['table_names_original']:
        test.tableNames.append(columnName)
    print("Schema", idx, ":", test.id, ":", test.tableNames)
    schemas[dbid] = test

tables = []
nl_queries = []

for idx, iquery in enumerate(queryDoc):
   # Initialize a blank array with the length of the number of tables
   tablesArray = [0] * len(schemas[iquery['db_id']].tableNames)
   # Get all the tokens from teh particular query we are looking at
   queryTokens = iquery['query_toks']

   # Search through all the tokens in the given query
   for idx, queryToken in enumerate(queryTokens):
      # Only look at tokens that come after the word "from," since those are the table names
      if (idx > 0) & (queryTokens[idx - 1].lower() == "from") | (queryTokens[idx - 1].lower() == "join"):
        # Search through all of the table names in the database we are looking at

        # TODO: Use spacy to tag all table names using named entity recognition (NER).
        for jdx, tableName in enumerate(schemas[iquery['db_id']].tableNames):
           # If the table names match, indicate it in tablesArray
           if tableName.lower() == queryToken.lower():
              tablesArray[jdx] = 1
   
   tables.append(tablesArray)
   print(tablesArray)
   #print(schemas[iquery['db_id']].id)
   print(iquery['query_toks'])

Schema 0 : perpetrator : ['perpetrator', 'people']
Schema 1 : college_2 : ['classroom', 'department', 'course', 'instructor', 'section', 'teaches', 'student', 'takes', 'advisor', 'time_slot', 'prereq']
Schema 2 : flight_company : ['airport', 'operate_company', 'flight']
Schema 3 : icfp_1 : ['Inst', 'Authors', 'Papers', 'Authorship']
Schema 4 : body_builder : ['body_builder', 'people']
Schema 5 : storm_record : ['storm', 'region', 'affected_region']
Schema 6 : pilot_record : ['aircraft', 'pilot', 'pilot_record']
Schema 7 : race_track : ['race', 'track']
Schema 8 : academic : ['author', 'conference', 'domain', 'domain_author', 'domain_conference', 'journal', 'domain_journal', 'keyword', 'domain_keyword', 'publication', 'domain_publication', 'organization', 'publication_keyword', 'writes', 'cite']
Schema 9 : department_store : ['Addresses', 'Staff', 'Suppliers', 'Department_Store_Chain', 'Customers', 'Products', 'Supplier_Addresses', 'Customer_Addresses', 'Customer_Orders', 'Department_St

In [5]:
def loadTrainDF(inputLines):
    wordSense = 0
    questions = []
    tables = []

    for line in inputLines:
        if line == "1\n":
            wordSense = 1
        elif line == "2\n":
            wordSense = 2
        elif (line != "\n") & (wordSense != 0):
            lines.append(line.strip())
            senses.append(wordSense)
    trainDF = pd.DataFrame(data={'text': lines,'sense': senses})
    trainDF['classification'] = trainDF['sense'].apply(lambda x: x - 1)
    return trainDF

In [ ]:
def get_sentence_embeddings(bert_preprocess_model, bert_encoder_model, sentences = []):
    
    text_preprocessed = bert_preprocess_model(sentences)
    return bert_encoder_model(text_preprocessed)['pooled_output']

In [ ]:
# Code Adopted and modified from YouTube video: https://www.youtube.com/watch?v=7kLi8u2dJz0&t=719s
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [ ]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

In [ ]:
# ! Code Adopted and Modified from https://github.com/eclarson/MachineLearningNotebooks and https://www.youtube.com/watch?v=hOCDJyZ6quA
# Notebook 13: RNN Basics
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input

# BERT Layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocessed_text)

# NN Layers
l = tf.keras.layers.Dropout(0.2, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

# Final Model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=3)
model.save('rubbish')

In [ ]:
rubbish_model = tf.keras.models.load_model('rubbish')

In [ ]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf

stopWordSet = nltk.corpus.stopwords.words('english')
stemmer = nltk.SnowballStemmer(language='english')
tokenizer = nltk.RegexpTokenizer(r"[a-zA-Z0-9]+")

def getTestData(inputfile):
    wordSense = 0
    lines = []
    senses = []

    for line in inputLines:
        if line == "1\n":
            wordSense = 1
        elif line == "2\n":
            wordSense = 2
        elif (line != "\n") & (wordSense != 0):
            lines.append(line.strip())
            senses.append(wordSense)
    trainDF = pd.DataFrame(data={'text': lines,'sense': senses})
    return trainDF

rubbishTestInput = open("testing/rubbish.txt", "r", encoding='utf-8')

lines = rubbishTestInput.readlines()
linesStripped = np.asarray([line.strip() for line in lines])

rubbish_predictions = rubbish_model.predict(linesStripped)
print(rubbish_predictions)

output_file = open('results/result_rubbish_Michael_Lennon.txt', "w", encoding='utf-8')

for prediction in rubbish_predictions:
    if prediction > 0.5:
        output_file.write('2\n')
    else:
        output_file.write('1\n')


# Rubbish Model.train()
rubbishTestInput.close()
output_file.close()